## Loading the data
## Clean the data
## Feature Enginnering
## Modelling
## Save model

<hr>

In [47]:
## Load the data

In [48]:
import pandas as pd

In [49]:
data = pd.read_csv('./data.csv')

In [50]:
## Clean the data

In [51]:
data.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country'],
      dtype='object')

In [52]:
data.drop(['sqft_living','sqft_lot','waterfront','view','condition','sqft_above','sqft_basement','street','city','statezip','country'],axis=1,inplace=True)

In [53]:
data.drop('date',axis=1,inplace=True)

In [54]:
data.head()

,price,bedrooms,bathrooms,floors,yr_built,yr_renovated
0,313000.0,3.0,1.50,1.5,1955,2005
1,2384000.0,5.0,2.50,2.0,1921,0
2,342000.0,3.0,2.00,1.0,1966,0
3,420000.0,3.0,2.25,1.0,1963,0
4,550000.0,4.0,2.50,1.0,1976,1992


In [55]:
## Feature Enginnering

In [56]:
def fe(data,col):
    print(len(data))
    max_no = data[col].quantile(0.99)
    min_no = data[col].quantile(0.05)
    data = data[data[col] > min_no]
    data = data[data[col] < max_no]
    print(len(data))
    return data

In [57]:
for col in list(data.columns):
    print(col)
    data = fe(data,'price')

price
4600
4316
bedrooms
4316
4056
bathrooms
4056
3808
floors
3808
3578
yr_built
3578
3362
yr_renovated
3362
3156


In [58]:
data.head()

,price,bedrooms,bathrooms,floors,yr_built,yr_renovated
2,342000.0,3.0,2.00,1.0,1966,0
3,420000.0,3.0,2.25,1.0,1963,0
4,550000.0,4.0,2.50,1.0,1976,1992
5,490000.0,2.0,1.00,1.0,1938,1994
6,335000.0,2.0,2.00,1.0,1976,0


In [59]:
X = data.drop('price',axis=1)
y = data['price']

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [62]:
len(X_train),len(X_test)

(2367, 789)

In [63]:
## Modelling

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim

In [65]:
class BaseLine_Model(nn.Module):
    def __init__(self,input_shape,output_shape):
        super().__init__()
        self.fc1 = nn.Linear(input_shape,32)
        self.fc2 = nn.Linear(32,64)
        self.fc3 = nn.Linear(64,128)
        self.fc4 = nn.Linear(128,64)
        self.fc5 = nn.Linear(64,output_shape)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = self.fc2(preds)
        preds = self.fc3(preds)
        preds = self.fc4(preds)
        preds = self.fc5(preds)
        return preds

In [66]:
EPOCHS = 100

In [67]:
import wandb

In [68]:
BATCH_SIZE = 32

In [69]:
PROJECT_NAME = 'House-Price-Pred'

In [70]:
from tqdm import tqdm

In [71]:
device = torch.device('cuda')

In [74]:
model = BaseLine_Model(5,1).to(device)

In [75]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.1)

In [123]:
def get_loss(criterion,X,y,model):
    preds = model(X.to(device))
    loss = criterion(preds,y)
    return loss.item()
def get_accuracy(X,y,model):
    correct = 0
    total = 0
    for i in range(len(X)):
        pred = model(X[i].float().to(device))
        pred.to(device)
        if pred[0] == y[i]:
            correct += 1
        total += 1
    if correct == 0:
        correct += 1
    return round(correct/total,3)

In [124]:
import numpy as np

In [125]:
X_train = torch.from_numpy(np.array(X_train))
y_train = torch.from_numpy(np.array(y_train))
X_test = torch.from_numpy(np.array(X_test))
y_test = torch.from_numpy(np.array(y_test))

In [126]:
get_accuracy(X_test,y_test,model)

0.001

In [127]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds.to(device)
        loss = criterion(preds.float(),y_batch.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})

  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: expected scalar type Float but found Double